In [1]:
import pandas as pd
import numpy as np
from evaluate import load
import copy
import random
import requests
from bert_score import score
from collections import defaultdict
import os

In [2]:
bertscore = load("bertscore")
squad_metric = load("squad")

In [3]:
data_path="../../data/"
output_path="../../outputs/"

In [4]:
def get_list_of_words():
    response = requests.get(
        'https://www.mit.edu/~ecprice/wordlist.10000',
        timeout=10
    )

    string_of_words = response.content.decode('utf-8')

    list_of_words = string_of_words.splitlines()

    return list_of_words

In [5]:
words = get_list_of_words()
#print(words)

In [6]:
random_word = random.choice(words)
print(random_word) 

usd


## bert

In [7]:
tasks_gpt3=pd.read_csv(output_path+"sg_ie/bert_tasks_simple.csv")

methods_gpt3=pd.read_csv(output_path+"sg_ie/bert_methods_simple.csv")

In [8]:
tasks_gpt3=tasks_gpt3.rename(columns={'clean_response':'task'})
tasks_gpt3=tasks_gpt3.loc[:,['ID','task']]

methods_gpt3=methods_gpt3.rename(columns={'clean_response':'method'})
methods_gpt3=methods_gpt3.loc[:,['ID','method']]

In [9]:
tasks_gpt3.task=tasks_gpt3.task.str.lstrip(' ')
methods_gpt3.method=methods_gpt3.method.str.lstrip(' ')

tasks_gpt3.task=tasks_gpt3.task.apply(lambda x:[x])
methods_gpt3.method=methods_gpt3.method.apply(lambda x:[x])

In [10]:
tasks_gpt3.shape

(263, 2)

## roberta

In [11]:
tasks_gpt3=pd.read_csv(output_path+"sg_ie/roberta_tasks_simple.csv")

methods_gpt3=pd.read_csv(output_path+"sg_ie/roberta_methods_simple.csv")

In [12]:
tasks_gpt3=tasks_gpt3.rename(columns={'clean_response':'task'})
tasks_gpt3=tasks_gpt3.loc[:,['ID','task']]

methods_gpt3=methods_gpt3.rename(columns={'clean_response':'method'})
methods_gpt3=methods_gpt3.loc[:,['ID','method']]

In [13]:
tasks_gpt3.task=tasks_gpt3.task.str.lstrip(' ')
methods_gpt3.method=methods_gpt3.method.str.lstrip(' ')

tasks_gpt3.task=tasks_gpt3.task.apply(lambda x:[x])
methods_gpt3.method=methods_gpt3.method.apply(lambda x:[x])

## deberta

In [14]:

tasks_gpt3=pd.read_csv(output_path+"sg_ie/deberta_tasks_simple3.csv")

methods_gpt3=pd.read_csv(output_path+"sg_ie/deberta_methods_simple3.csv")

In [15]:
tasks_gpt3=tasks_gpt3.rename(columns={'clean_response':'task'})
tasks_gpt3=tasks_gpt3.loc[:,['ID','task']]

methods_gpt3=methods_gpt3.rename(columns={'clean_response':'method'})
methods_gpt3=methods_gpt3.loc[:,['ID','method']]

In [16]:
tasks_gpt3.task=tasks_gpt3.task.str.lstrip(' ')
methods_gpt3.method=methods_gpt3.method.str.lstrip(' ')

tasks_gpt3.task=tasks_gpt3.task.apply(lambda x:[x])
methods_gpt3.method=methods_gpt3.method.apply(lambda x:[x])

## davinci 3

In [19]:
tasks_gpt3=pd.read_csv(output_path+"sg_ie/gpt3/GPT3_responses_task_davinci3.csv")

methods_gpt3=pd.read_csv(output_path+"sg_ie/gpt3/GPT3_responses_method2_davinci3.csv")

In [20]:
tasks_gpt3=tasks_gpt3.rename(columns={'clean_response':'task'})
tasks_gpt3=tasks_gpt3.loc[:,['ID','task']]

methods_gpt3=methods_gpt3.rename(columns={'clean_response':'method'})
methods_gpt3=methods_gpt3.loc[:,['ID','method']]

In [21]:
tasks_gpt3.task=tasks_gpt3.task.str.lstrip(' ')
methods_gpt3.method=methods_gpt3.method.str.lstrip(' ')

tasks_gpt3.task=tasks_gpt3.task.apply(lambda x:[x])
methods_gpt3.method=methods_gpt3.method.apply(lambda x:[x])

## random

In [22]:
for i,d in tasks_gpt3.iterrows():
    tasks_gpt3.at[i,'task']=random.choices(words,k=2)
for i,d in tasks_gpt3.iterrows():
    methods_gpt3.at[i,'method']=random.choices(words,k=2)

## davinci 2 multiple

In [23]:
methods_gpt3=pd.read_csv(output_path+"sg_ie/gpt3/GPT3_responses_method2_cleaned_f.csv")
tasks_gpt3=pd.read_csv(output_path+"sg_ie/gpt3/GPT3_responses_task_cleaned_f2.csv")

methods_gpt3=methods_gpt3.groupby('ID')['methods'].apply(list).reset_index().rename(columns={'methods':'method'})
tasks_gpt3=tasks_gpt3.groupby('ID')['tasks'].apply(list).reset_index().rename(columns={'tasks':'task'})

# davinci 2

In [25]:
tasks_gpt3=pd.read_csv(output_path+"sg_ie/gpt3/GPT3_responses_task_davinci2_coarse2.csv")

methods_gpt3=pd.read_csv(output_path+"sg_ie/gpt3/GPT3_responses_method_davinci2_coarse2.csv")

tasks_gpt3=tasks_gpt3.rename(columns={'clean_response':'task'})
tasks_gpt3=tasks_gpt3.loc[:,['ID','task']]

methods_gpt3=methods_gpt3.rename(columns={'clean_response':'method'})
methods_gpt3=methods_gpt3.loc[:,['ID','method']]

tasks_gpt3.task=tasks_gpt3.task.str.lstrip(' ')
methods_gpt3.method=methods_gpt3.method.str.lstrip(' ')

tasks_gpt3.task=tasks_gpt3.task.apply(lambda x:[x])
methods_gpt3.method=methods_gpt3.method.apply(lambda x:[x])

## gold data 1

In [8]:
gold_usr1=pd.read_json(output_path+"sg_ie/annotation_20230114.json")

In [9]:
obs=[]
for i,d in gold_usr1.iterrows():
    response={}
    response['task']=[]
    response['method']=[]
    response['SG_element_in_task']=[]
    for e in d['annotations'][0]['result']:
        if e['from_name']=='label':
            if e['value']['labels'][0] in ['task']:
                response['task'].append(e['value']['text'])
            elif e['value']['labels'][0] in ['method']:
                response['method'].append(e['value']['text'])
            elif e['value']['labels'][0]=='SG_element_in_task':
                response['SG_element_in_task'].append(e['value']['text'])
        elif e['from_name']=='method_c':
            response['method'].append(e['value']['text'][0])
        elif e['from_name']=='task_c':
            response['task'].append(e['value']['text'][0])
    response['ID']=d['data']['ID']
    obs.append(response)

In [10]:
df_usr1=pd.DataFrame(obs)

## gold data

In [11]:
test_set_labeled=pd.read_csv(output_path+"general/test_set_final.csv")

test_set_labeled=test_set_labeled.iloc[:2000]

In [12]:
gold=pd.read_json(output_path+"sg_ie/final_annotation_user2.json")

In [13]:
obs=[]
for i,d in gold.iterrows():
    response={}
    response['task']=[]
    response['method']=[]
    response['SG_element_in_task']=[]
    for e in d['annotations'][0]['result']:
        if e['from_name']=='label':
            if e['value']['labels'][0] in ['task']:
                response['task'].append(e['value']['text'])
            elif e['value']['labels'][0] in ['method']:
                response['method'].append(e['value']['text'])
            elif e['value']['labels'][0]=='SG_element_in_task':
                response['SG_element_in_task'].append(e['value']['text'])
        elif e['from_name']=='method_c':
            response['method'].append(e['value']['text'][0])
        elif e['from_name']=='task_c':
            response['task'].append(e['value']['text'][0])
    response['ID']=d['data']['ID']
    obs.append(response)

In [14]:
df=pd.DataFrame(obs)

In [15]:
df=df.loc[df.ID.isin(test_set_labeled.ID.unique())]

In [16]:
#df=df.loc[df.ID.isin(df_usr1.ID.unique())]

In [17]:
df=df.rename(columns={'task':'gold_tasks','method':'gold_methods'})

In [18]:
df.shape

(251, 4)

In [19]:
df_final=df.merge(methods_gpt3,on=['ID']).merge(tasks_gpt3,on=['ID'])

In [20]:
df_final.shape

(251, 6)

In [21]:
sq_methods_pred=[]
sq_methods_ref=[]
sq_tasks_pred=[]
sq_tasks_ref=[]
for i,d in df_final.iterrows():
    preds_t=[]
    preds_m=[]
    refs_t=[]
    refs_m=[]
    pred_method={}
    ref_method={}
    pred_task={}
    ref_task={}
    refs_t=copy.deepcopy(list(set(d['gold_tasks'])))
    #refs_t.append(', '.join(d['gold_tasks']))
    preds_t=d['task']
    results_t = bertscore.compute(predictions=[', '.join(preds_t)], references=[', '.join(refs_t)],  model_type="distilbert-base-uncased")
    refs_m=copy.deepcopy(list(set(d['gold_methods'])))
    preds_m=d['method']
    #refs_m.append(', '.join(d['gold_methods']))
    results_m = bertscore.compute(predictions=[', '.join(preds_m)], references=[', '.join(refs_m)],  model_type="distilbert-base-uncased")
    pred_method['prediction_text']=', '.join(d['method'])
    pred_method['id']=d['ID']
    ref_method['answer_start']=[1]
    ref_method['text']=[copy.deepcopy(', '.join(list(set(d['gold_methods']))))]
    #ref_method['text'].append(', '.join(d['gold_methods']))
    ref_dict_m={'answers':ref_method,'id':d['ID']}
    sq_methods_pred.append(pred_method)
    sq_methods_ref.append(ref_dict_m)
    pred_task['prediction_text']=', '.join(d['task'])
    pred_task['id']=d['ID']
    ref_task['answer_start']=[1]
    ref_task['text']=[copy.deepcopy(', '.join(list(set(d['gold_tasks']))))]
    #ref_task['text'].append(', '.join(d['gold_tasks']))
    ref_dict={'answers':ref_task,'id':d['ID']}
    sq_tasks_pred.append(pred_task)
    sq_tasks_ref.append(ref_dict)
    df_final.at[i,'bert_score_task']=results_t['f1'][0]
    df_final.at[i,'bert_score_method']=results_m['f1'][0]
    df_final.at[i,'bert_score_task_p']=results_t['precision'][0]
    df_final.at[i,'bert_score_method_p']=results_m['precision'][0]
    df_final.at[i,'bert_score_task_r']=results_t['recall'][0]
    df_final.at[i,'bert_score_method_r']=results_m['recall'][0]
    #if i==3:
    #    break

## new refs

In [22]:
## test
df_final.loc[:,['bert_score_task','bert_score_task_p','bert_score_task_r','bert_score_method','bert_score_method_p','bert_score_method_r']].describe()

,bert_score_task,bert_score_task_p,bert_score_task_r,bert_score_method,bert_score_method_p,bert_score_method_r
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.846537,0.849782,0.845829,0.779509,0.793129,0.770018
std,0.126584,0.130700,0.130282,0.209024,0.216502,0.209427
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.770662,0.764325,0.765881,0.720510,0.722997,0.715970
50%,0.844353,0.854298,0.840995,0.800789,0.802708,0.780216
75%,0.927946,0.956243,0.955311,0.895158,0.944155,0.885381
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
## bert
df_final.loc[:,['bert_score_task','bert_score_task_p','bert_score_task_r','bert_score_method','bert_score_method_p','bert_score_method_r']].describe()

,bert_score_task,bert_score_task_p,bert_score_task_r,bert_score_method,bert_score_method_p,bert_score_method_r
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.822901,0.816570,0.831548,0.750439,0.755597,0.748608
std,0.130174,0.133811,0.133300,0.203076,0.210065,0.203786
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.738739,0.725715,0.746157,0.701617,0.696307,0.700566
50%,0.807494,0.795717,0.812928,0.755700,0.747851,0.755346
75%,0.916532,0.930807,0.953357,0.873388,0.901863,0.862068
max,1.000001,1.000001,1.000001,1.000000,1.000000,1.000000


In [22]:
## roberta
df_final.loc[:,['bert_score_task','bert_score_task_p','bert_score_task_r','bert_score_method','bert_score_method_p','bert_score_method_r']].describe()

,bert_score_task,bert_score_task_p,bert_score_task_r,bert_score_method,bert_score_method_p,bert_score_method_r
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.813914,0.817180,0.813370,0.764667,0.774199,0.758573
std,0.131256,0.134276,0.136525,0.208108,0.216194,0.207169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.724325,0.723986,0.725481,0.707466,0.705819,0.711994
50%,0.801688,0.797297,0.790870,0.762037,0.772338,0.763397
75%,0.903407,0.918530,0.921604,0.885452,0.934655,0.867019
max,1.000001,1.000001,1.000001,1.000000,1.000000,1.000000


In [22]:
## deberta
df_final.loc[:,['bert_score_task','bert_score_task_p','bert_score_task_r','bert_score_method','bert_score_method_p','bert_score_method_r']].describe()

,bert_score_task,bert_score_task_p,bert_score_task_r,bert_score_method,bert_score_method_p,bert_score_method_r
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.803627,0.806096,0.803534,0.756241,0.751977,0.763300
std,0.125885,0.128406,0.131280,0.203369,0.206775,0.206073
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.721920,0.725548,0.719768,0.709259,0.690596,0.716571
50%,0.788053,0.781762,0.790225,0.761000,0.765336,0.767456
75%,0.889561,0.894909,0.906817,0.882352,0.888646,0.885876
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
## davinci 3
df_final.loc[:,['bert_score_task','bert_score_task_p','bert_score_task_r','bert_score_method','bert_score_method_p','bert_score_method_r']].describe()

,bert_score_task,bert_score_task_p,bert_score_task_r,bert_score_method,bert_score_method_p,bert_score_method_r
count,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000
mean,0.864270,0.856361,0.875250,0.781984,0.789812,0.777464
std,0.101583,0.111278,0.102421,0.182796,0.191288,0.181665
min,0.690875,0.653883,0.693537,0.000000,0.000000,0.000000
25%,0.766959,0.760829,0.773734,0.712723,0.714117,0.715195
50%,0.854298,0.837344,0.874330,0.784332,0.786143,0.760902
75%,0.966874,0.977267,0.979164,0.890677,0.930055,0.898305
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
## davinci 2 new
df_final.loc[:,['bert_score_task','bert_score_task_p','bert_score_task_r','bert_score_method','bert_score_method_p','bert_score_method_r']].describe()

,bert_score_task,bert_score_task_p,bert_score_task_r,bert_score_method,bert_score_method_p,bert_score_method_r
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.846537,0.849782,0.845829,0.779509,0.793129,0.770018
std,0.126584,0.130700,0.130282,0.209024,0.216502,0.209427
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.770662,0.764325,0.765881,0.720510,0.722997,0.715970
50%,0.844353,0.854298,0.840995,0.800789,0.802708,0.780216
75%,0.927946,0.956243,0.955311,0.895158,0.944155,0.885381
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
## random
df_final.loc[:,['bert_score_task','bert_score_task_p','bert_score_task_r','bert_score_method','bert_score_method_p','bert_score_method_r']].describe()

,bert_score_task,bert_score_task_p,bert_score_task_r,bert_score_method,bert_score_method_p,bert_score_method_r
count,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000
mean,0.688637,0.695748,0.682294,0.670660,0.680255,0.662417
std,0.033163,0.034766,0.037878,0.132141,0.135619,0.131989
min,0.612957,0.615123,0.596983,0.000000,0.000000,0.000000
25%,0.667830,0.671548,0.654122,0.661428,0.665533,0.649383
50%,0.689084,0.695649,0.680825,0.691087,0.699459,0.681993
75%,0.714860,0.717431,0.713611,0.716944,0.728929,0.713543
max,0.759717,0.780435,0.765326,0.810814,0.823473,0.798538


## test  single

In [24]:
results_t = squad_metric.compute(predictions=sq_tasks_pred, references=sq_tasks_ref)
results_t

{'exact_match': 19.12350597609562, 'f1': 43.94038040662075}

In [25]:
results_m = squad_metric.compute(predictions=sq_methods_pred, references=sq_methods_ref)
results_m

{'exact_match': 11.55378486055777, 'f1': 32.0457783612836}

## bert  single

In [24]:
results_t = squad_metric.compute(predictions=sq_tasks_pred, references=sq_tasks_ref)
results_t

{'exact_match': 15.936254980079681, 'f1': 35.07567593406898}

In [25]:
results_m = squad_metric.compute(predictions=sq_methods_pred, references=sq_methods_ref)
results_m

{'exact_match': 4.382470119521912, 'f1': 23.08060231308414}

## roberta  single

In [23]:
results_t = squad_metric.compute(predictions=sq_tasks_pred, references=sq_tasks_ref)
results_t

{'exact_match': 14.741035856573705, 'f1': 33.433708571979025}

In [24]:
results_m = squad_metric.compute(predictions=sq_methods_pred, references=sq_methods_ref)
results_m

{'exact_match': 9.56175298804781, 'f1': 27.29846299752432}

## deberta  single

In [23]:
results_t = squad_metric.compute(predictions=sq_tasks_pred, references=sq_tasks_ref)
results_t

{'exact_match': 11.155378486055778, 'f1': 31.284398757848614}

In [24]:
results_m = squad_metric.compute(predictions=sq_methods_pred, references=sq_methods_ref)
results_m

{'exact_match': 4.780876494023905, 'f1': 26.05364464509925}

## davinci 3 single

In [21]:
results_t = squad_metric.compute(predictions=sq_tasks_pred, references=sq_tasks_ref)
results_t

{'exact_match': 23.64864864864865, 'f1': 48.06050236197295}

In [22]:
results_m = squad_metric.compute(predictions=sq_methods_pred, references=sq_methods_ref)
results_m

{'exact_match': 10.135135135135135, 'f1': 29.432106653468878}

## davinci2 

In [26]:
results_t = squad_metric.compute(predictions=sq_tasks_pred, references=sq_tasks_ref)
results_t

{'exact_match': 19.12350597609562, 'f1': 43.94038040662075}

In [27]:
results_m = squad_metric.compute(predictions=sq_methods_pred, references=sq_methods_ref)
results_m

{'exact_match': 11.55378486055777, 'f1': 32.0457783612836}

## random

In [23]:
results_t = squad_metric.compute(predictions=sq_tasks_pred, references=sq_tasks_ref)
results_t

{'exact_match': 0.0, 'f1': 0.0}

In [24]:
results_m = squad_metric.compute(predictions=sq_methods_pred, references=sq_methods_ref)
results_m

{'exact_match': 0.0, 'f1': 0.0}

## davinci2 multiple

In [16]:
results_t = squad_metric.compute(predictions=sq_tasks_pred, references=sq_tasks_ref)
results_t

{'exact_match': 0.970873786407767, 'f1': 28.917341146067596}

In [17]:
results_m = squad_metric.compute(predictions=sq_methods_pred, references=sq_methods_ref)
results_m

{'exact_match': 0.970873786407767, 'f1': 29.629456304213576}